# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [2]:
import pandas as pd
import requests

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [3]:
# 1) Seu código aqui
url = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl'
r = requests.get(url)
r.status_code


200

In [4]:
data = r.json()
df = pd.DataFrame(data['items'])
df_rj = df[(df['uf'] == 'RJ') &
           (df['tipo_interessado'] == 'Município')]
df_rj.head()

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
110,13594,Município,Rio de Janeiro,3304557,RJ,None,Arquivado,19405.000032/2006-84,2006-07-20T03:00:00Z,Operação contratual interna,Saneamento básico,Instituição Financeira Nacional,Caixa Econômica Federal,Real,1.914638e+07,0,0,26/06/2007
133,12747,Município,Niterói,3303302,RJ,None,Encaminhado à PGFN com manifestação técnica fa...,17944.001779/2006-27,2013-12-04T02:00:00Z,Operação contratual externa (com garantia da U...,Multissetorial,Instituição Financeira Internacional,Banco Interamericano de Desenvolvimento,Dólar dos EUA,2.647000e+07,1,0,19/12/2013
137,13677,Município,Barra Mansa,3300407,RJ,None,Deferido,19405.000049/2005-51,2006-05-05T03:00:00Z,Operação contratual interna,PMAT,Instituição Financeira Nacional,Banco Nacional de Desenvolvimento Econômico e ...,Real,2.669010e+06,0,0,15/05/2006
156,13690,Município,Rio de Janeiro,3304557,RJ,None,Arquivado,19405.000052/2003-11,2005-05-01T03:00:00Z,Operação contratual externa (com garantia da U...,Multissetorial,Instituição Financeira Internacional,Banco Internacional para Reconstrução e Desenv...,Dólar dos EUA,1.779240e+08,0,0,17/08/2006
453,42252,Município,Itatiaia,3302254,RJ,PVL02.000042/2020-82,Encaminhado à PGFN com manifestação técnica fa...,17944.101489/2020-31,2020-05-19T13:29:59Z,Operação contratual interna (com garantia da U...,Infraestrutura,Instituição Financeira Nacional,Banco do Brasil S/A,Real,3.000000e+07,1,1,21/05/2020


In [5]:
# 2) Seu código aqui
status = df_rj['status'].value_counts()
print(status[:3])

status
Deferido        29
Regularizado    10
Arquivado        9
Name: count, dtype: int64


In [6]:
# 3) Seu código aqui
df_rj['data_status'] = df_rj['data_status'].astype(str)
df_rj['ano_status'] = df_rj['data_status'].str.slice(-4)
df_rj['ano_status'].head()

C:\Users\jpbda\AppData\Local\Temp\ipykernel_13808\1337915019.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rj['data_status'] = df_rj['data_status'].astype(str)
C:\Users\jpbda\AppData\Local\Temp\ipykernel_13808\1337915019.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rj['ano_status'] = df_rj['data_status'].str.slice(-4)


110    2007
133    2013
137    2006
156    2006
453    2020
Name: ano_status, dtype: object

In [7]:
df_rj['ano_status'].value_counts()

ano_status
2007    12
2023     9
2020     6
2008     6
2019     4
2012     3
2006     3
2002     3
2014     3
2013     3
2011     3
2016     2
2017     2
2010     2
2021     2
2015     1
2018     1
2009     1
2022     1
Name: count, dtype: int64

####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [8]:
#1) Seu código aqui
def filtro_uf_tipo_interessado(uf, tipo_interessado):
    uf = uf.upper()
    tipo_interessado = tipo_interessado.capitalize()
    df_filtro = df[(df['uf'] == uf) &
                (df['tipo_interessado'] == tipo_interessado)]
    
    return df_filtro


In [9]:
# 2) Seu código aqui
uf = 'MG'
tipo_interessado = 'Estado'

df_filtro = filtro_uf_tipo_interessado(uf, tipo_interessado)
df_arquivado = df_filtro[df_filtro['status'] == 'Arquivado por descurso de prazo']

In [10]:
df[(df['uf'] == 'MG') & (df['tipo_interessado'] == 'Estado') & (df['status'] == 'Arquivado por descurso de prazo')]

#Não existe nenhum item que atenda a essas condições

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status


In [11]:
# 3) Seu código aqui
df_filtro = filtro_uf_tipo_interessado('ba', 'municíPio')
df_filtro['interessado'].value_counts()

interessado
Bom Jesus da Lapa         4
Barreiras                 4
Luís Eduardo Magalhães    3
América Dourada           3
Feira de Santana          3
                         ..
Abaíra                    1
Salvador                  1
Santa Maria da Vitória    1
São Félix                 1
Ubaíra                    1
Name: count, Length: 137, dtype: int64

In [12]:
# 4) Seu código aqui
df_filtro = filtro_uf_tipo_interessado('ba', 'estado')
df_filtro.to_csv('ba_estado.csv')